# LUXOR v7.1 AGGRESSIVE - Performance Analysis
Interactive notebook for analyzing backtest results and strategy performance

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('..')

from src.backtester import run_backtest
from src.utils import load_config, print_performance_summary

%matplotlib inline
sns.set_style('darkgrid')

## 1. Load Configuration and Run Backtest

In [ ]:
config = load_config('../config.json')
trades_df, data_df, stats = run_backtest(config)
print_performance_summary(stats)

## 2. Trade Distribution Analysis

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# P&L distribution
axes[0, 0].hist(trades_df['pnl_percent'], bins=30, edgecolor='black')
axes[0, 0].set_title('P&L Distribution')
axes[0, 0].set_xlabel('P&L %')
axes[0, 0].axvline(0, color='red', linestyle='--')

# Direction breakdown
direction_pnl = trades_df.groupby('direction')['pnl_percent'].sum()
axes[0, 1].bar(direction_pnl.index, direction_pnl.values)
axes[0, 1].set_title('P&L by Direction')
axes[0, 1].set_ylabel('Total P&L %')

# Score vs P&L
axes[1, 0].scatter(trades_df['score'], trades_df['pnl_percent'], alpha=0.6)
axes[1, 0].set_title('Score vs P&L')
axes[1, 0].set_xlabel('Entry Score')
axes[1, 0].set_ylabel('P&L %')
axes[1, 0].axhline(0, color='red', linestyle='--')

# Cumulative P&L
trades_df['cumulative_pnl'] = trades_df['pnl_percent'].cumsum()
axes[1, 1].plot(trades_df['cumulative_pnl'])
axes[1, 1].set_title('Equity Curve')
axes[1, 1].set_xlabel('Trade #')
axes[1, 1].set_ylabel('Cumulative P&L %')
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

## 3. Top Winners and Losers

In [ ]:
print("\n🏆 TOP 5 WINNERS")
print(trades_df.nlargest(5, 'pnl_percent')[['entry_date', 'direction', 'pnl_percent', 'score']])

print("\n❌ TOP 5 LOSERS")
print(trades_df.nsmallest(5, 'pnl_percent')[['entry_date', 'direction', 'pnl_percent', 'score', 'exit_reason']])

## 4. Monthly Performance

In [ ]:
trades_df['entry_month'] = pd.to_datetime(trades_df['entry_date']).dt.to_period('M')
monthly_pnl = trades_df.groupby('entry_month')['pnl_percent'].sum()

plt.figure(figsize=(15, 5))
monthly_pnl.plot(kind='bar')
plt.title('Monthly P&L')
plt.xlabel('Month')
plt.ylabel('P&L %')
plt.axhline(0, color='red', linestyle='--')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 5. Exit Reason Analysis

In [ ]:
exit_analysis = trades_df.groupby('exit_reason').agg({
    'pnl_percent': ['count', 'mean', 'sum']
}).round(2)

print("\nEXIT REASON ANALYSIS")
print(exit_analysis)